# SP ENERGY NETWORKS Power Cuts - API Exploration

**Goal:** Fetch and explore live power outage data from SP Energy Network. 
## API Endpoint <br>  
**CKAN DataStore API:**  https://spenergynetworks.opendatasoft.com/api/explore/v2.1/catalog/datasets/distribution-network-live-outages/records

## 0. Setup And Imports

In [13]:
# Import Libaries
import requests
import pandas as pd
import json
from datetime import datetime
from pprint import pprint
from dotenv import load_dotenv
import os

# Display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 1. API Configurations 

In [14]:
# Base URL and dataset identifier
BASE_URL = "https://spenergynetworks.opendatasoft.com/api/explore/v2.1"
DATASET_ID = "distribution-network-live-outages"

# Construct the full endpoint
API_ENDPOINT = f"{BASE_URL}/catalog/datasets/{DATASET_ID}/records"

print(f"API Endpoint: {API_ENDPOINT}")

API Endpoint: https://spenergynetworks.opendatasoft.com/api/explore/v2.1/catalog/datasets/distribution-network-live-outages/records


## 2. Dataset Metadata

In [15]:
# Get dataset metadata
metadata_url = f"{BASE_URL}/catalog/datasets/{DATASET_ID}"

response = requests.get(metadata_url)

if response.status_code == 200:
    metadata = response.json()

    print("=" * 80)
    print("DATASET METADATA")
    print("=" * 80)
    print(f"Dataset ID: {metadata.get('dataset_id')}")
    print(f"Title: {metadata['metas']['default'].get('title')}")
    print(
        f"Description: {metadata['metas']['default'].get('description', 'N/A')[:200]}...")
    print(f"\nPublisher: {metadata['metas']['default'].get('publisher')}")
    print(
        f"Modified: {metadata.get('metas', {}).get('default', {}).get('modified')}")
    print(
        f"Update Frequency: {metadata['metas']['default'].get('frequency', 'N/A')}")
    print(
        f"Total Records: {metadata.get('metas', {}).get('default', {}).get('records_count', 'N/A')}")
    print(f"\nThemes: {metadata['metas']['default'].get('theme', [])}")
    print(f"Keywords: {metadata['metas']['default'].get('keyword', [])}")

else:
    print(f"Error fetching metadata: {response.status_code}")

DATASET METADATA
Dataset ID: distribution-network-live-outages
Title: Distribution Network Live Outages
Description: <p>This "<b>Distribution Network Live Outages</b>" data table contains live outage data for SPEN's distribution network, covering both Low Voltage (LV) and High Voltage (HV) networks. The data is coll...

Publisher: SP Energy Networks SC389555
Modified: 2025-11-20T12:45:12+00:00
Update Frequency: N/A
Total Records: 31

Themes: ['Network Usage']
Keywords: ['Outages', 'Faults', 'Live']


## 3. Dataset Schema Columns

In [16]:
if response.status_code == 200:
    print("\n" + "=" * 80)
    print("DATASET SCHEMA - AVAILABLE COLUMNS")
    print("=" * 80)

    # Extract field information
    fields = metadata.get('fields', [])

    print(f"\nTotal Fields: {len(fields)}\n")

    # Create a DataFrame for better display
    schema_data = []
    for field in fields:
        schema_data.append({
            'Field Name': field.get('name'),
            'Label': field.get('label'),
            'Type': field.get('type'),
            'Description': field.get('description', 'N/A')[:60] + '...' if field.get('description') and len(field.get('description', '')) > 60 else field.get('description', 'N/A')
        })

    schema_df = pd.DataFrame(schema_data)
    print(schema_df.to_string(index=False))


DATASET SCHEMA - AVAILABLE COLUMNS

Total Fields: 10

               Field Name                         Label     Type                                                     Description
                 fault_id                      Fault ID     text            A unique identifier assigned to each fault incident.
             licence_area                   DNO Licence     text  The license number of the Distribution Network Operator res...
                   region                        Region     text  The geographical area or region where the outage incident t...
                   status               Incident Status     text  The current status of the outage incident (e.g., ongoing, r...
                  planned                       Planned  boolean              Indicates whether the outage was planned (yes/no).
planned_outage_start_date          Planned Outage Start datetime          The scheduled start date and time for planned outages.
   date_of_reported_fault           Fault 

## 4. Fetch Sample Data

In [17]:
print("\n" + "=" * 80)
print("FETCHING SAMPLE DATA (First 5 records)")
print("=" * 80)

load_dotenv()

# Get API key from environment
API_KEY = os.getenv('SP_ENERGY_API_KEY')

headers = {
    "Authorization": f"Apikey {API_KEY}"
}

# Parameters for the API request
params = {
    "limit": 100,  # Get first 100 records
    "timezone": "Europe/London"
}

# Make the API request
response = requests.get(API_ENDPOINT, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    
    print(f"\nAPI Response Status: {response.status_code} - Success!")
    print(f"Total Records Available: {data.get('total_count', 'N/A')}")
    print(f"Records Retrieved: {len(data.get('results', []))}")
    
    # Check if we have results
    if data.get('results'):
        print(f"\n✓ Successfully retrieved sample data!")
    else:
        print("\n⚠ No records found in the dataset")
else:
    print(f"\n✗ Error: {response.status_code}")
    print(response.text)


FETCHING SAMPLE DATA (First 5 records)

API Response Status: 200 - Success!
Total Records Available: 31
Records Retrieved: 31

✓ Successfully retrieved sample data!


## 5. Display Sample Data

In [18]:
if response.status_code == 200 and data.get('results'):
    print("\n" + "=" * 80)
    print("SAMPLE DATA - DATAFRAME VIEW")
    print("=" * 80)

    # Extract fields from results
    records_list = []
    for record in data['results']:
        # Opendatasoft often nests fields differently
        # Try multiple possible structures
        if 'fields' in record:
            records_list.append(record['fields'])
        elif 'record' in record and 'fields' in record['record']:
            records_list.append(record['record']['fields'])
        else:
            records_list.append(record)

    # Convert to DataFrame
    df = pd.DataFrame(records_list)

    print(f"\nShape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

    print("\n" + "-" * 80)
    print("FIRST 5 RECORDS:")
    print("-" * 80)
    display(df.head())

    print("\n" + "=" * 80)

    # Check unique values in Status and Planned columns
    print("\n=== Unique Values Analysis ===")

    if 'status' in df.columns:
        print(f"\nUnique values in 'status' column:")
        print(df['status'].unique())
        print(f"Value counts: {df['status'].value_counts().to_dict()}")
    else:
        print("\n⚠️ 'status' column not found in dataframe")

    if 'planned' in df.columns:
        print(f"\nUnique values in 'planned' column:")
        print(df['planned'].unique())
        print(f"Value counts: {df['planned'].value_counts().to_dict()}")
    else:
        print("\n⚠️ 'planned' column not found in dataframe")

    # Check data types
    if 'status' in df.columns:
        print(f"\nData type of 'status': {df['status'].dtype}")
    if 'planned' in df.columns:
        print(f"Data type of 'planned': {df['planned'].dtype}")

    # See sample combinations
    print(f"\nSample Status + Planned combinations:")
    if 'status' in df.columns and 'planned' in df.columns:
        print(df[['status', 'planned']].drop_duplicates().head(10))
    elif 'status' in df.columns:
        print(df[['status']].drop_duplicates().head(10))
    elif 'planned' in df.columns:
        print(df[['planned']].drop_duplicates().head(10))


SAMPLE DATA - DATAFRAME VIEW

Shape: (31, 10)
Columns: ['fault_id', 'licence_area', 'region', 'status', 'planned', 'planned_outage_start_date', 'date_of_reported_fault', 'etr', 'voltage', 'postcode_sector']

--------------------------------------------------------------------------------
FIRST 5 RECORDS:
--------------------------------------------------------------------------------


,fault_id,licence_area,region,status,planned,planned_outage_start_date,date_of_reported_fault,etr,voltage,postcode_sector
0,INCD-906201-f,SP Distribution,STIRLING,Awaiting,True,2025-11-20T09:00:00+00:00,2025-10-31T10:23:31+00:00,2025-11-20T15:30:00+00:00,HV,"[FK8 3, FK7 9]"
1,INCD-906759-f,SP Manweb,RUNCORN,Awaiting,True,2025-11-20T09:00:00+00:00,2025-11-05T10:44:26+00:00,2025-11-20T16:00:00+00:00,HV,[CW9 6]
2,INCD-906849-f,SP Manweb,OSWESTRY NORTH,Awaiting,True,2025-11-20T09:00:00+00:00,2025-11-05T15:58:14+00:00,2025-11-20T13:00:00+00:00,LV,[SY11 3]
3,INCD-907191-f,SP Manweb,GWYNEDD MENAI,Awaiting,True,2025-11-20T09:00:00+00:00,2025-11-07T09:28:52+00:00,2025-11-20T16:00:00+00:00,HV,"[LL65 3, LL65 4, LL71 7]"
4,INCD-2389466-i,SP Manweb,GWYNEDD ERYRI,Awaiting,False,None,2025-11-19T21:17:05+00:00,2025-11-20T16:00:00+00:00,HV,"[LL53 8, LL53 6]"




=== Unique Values Analysis ===

Unique values in 'status' column:
['Awaiting' 'In Progress']
Value counts: {'Awaiting': 30, 'In Progress': 1}

Unique values in 'planned' column:
[ True False]
Value counts: {True: 18, False: 13}

Data type of 'status': object
Data type of 'planned': bool

Sample Status + Planned combinations:
         status  planned
0      Awaiting     True
4      Awaiting    False
28  In Progress    False
